# Dogs breeds

https://youtu.be/JNxcznsrRb8?t=1h31m8s

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.set_device(0)

In [4]:
PATH = "/home/embsysa/fastai/pytorch_classifiers-master/data/"
sz = 224
arch = resnet34
bs = 24

## Initial exploration

In [5]:
import os
data_path = "/home/embsysa/fastai/pytorch_classifiers-master/data"
os.chdir(data_path)

In [6]:
!ls {PATH}

label_csv_name = 'Emotion_labels.csv'
label_df = pd.read_csv(label_csv_name)


 all_mask.zip				  Major_Background_labels.csv
 Background_color_allmultilabel.csv	  Major_Graphics_labels.csv
 Background_color_allmultilabel.csv.csv   Major_Graphics_labels.csv.csv
 Backup					  Major_Graphics_labels_final.csv
'change file type.ipynb'		  old_Major_Background_labels.csv
 Emotion_labels.csv			  test
 Emotion_labels.csv.csv			  tmp
'Final data Folders'			  train
'Final data Folders.zip'		  train_with_mask


In [7]:
images = [each.split(".jpg")[0] for each in os.listdir(data_path+"/train")]

In [8]:
label_df.head()

,Image_index,Emotion
0,28,P
1,40,N
2,698,P
3,769,P
4,786,N


In [9]:
label_df = label_df[label_df.Image_index.isin(images)]
label_df.to_csv(label_csv_name+'.csv', index=False)

In [10]:
label_csv_name

'Emotion_labels.csv'

In [11]:
label_csv = f'{PATH}'+label_csv_name+'.csv'
n = len(label_df)# header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [12]:
label_df.columns = label_df.columns.str.replace("-","_")
label_df.head()
#label_df.columns = ['id', 'color']

label_df.shape

(9946, 2)

In [13]:
#label_df.pivot_table(index="color", aggfunc=len).sort_values('id', ascending=False)
import numpy as np
np.max(val_idxs)

9944

In [14]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.2)
data = ImageClassifierData.from_csv(PATH, 'train', label_csv, test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

fn = PATH + data.trn_ds.fnames[0];
print(fn)

data.trn_ds.fnames[0]

img = PIL.Image.open(fn); img

img.size

/home/embsysa/fastai/pytorch_classifiers-master/data/train/40.jpg


(512, 512)

In [15]:
len(data.val_y)

1989

In [16]:
size_d = {k: PIL.Image.open(PATH + k).size for k in data.trn_ds.fnames}

In [17]:
row_sz, col_sz = list(zip(*size_d.values()))

In [18]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

## Initial model

In [19]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.2)
    data = ImageClassifierData.from_csv(PATH, 'train', label_csv, test_name='test',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms,bs = bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    return data if sz > 300 else data.resize(256, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time

#Source:   
#    def resize(self, targ, new_path):
#        new_ds = []
#        dls = [self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl]
#        if self.test_dl: dls += [self.test_dl, self.test_aug_dl]
#        else: dls += [None,None]
#        t = tqdm_notebook(dls)
#        for dl in t: new_ds.append(self.resized(dl, targ, new_path))
#        t.close()
#        return self.__class__(new_ds[0].path, new_ds, self.bs, self.num_workers, self.classes)
#File:      ~/fastai/courses/dl1/fastai/dataset.py

### Precompute

In [20]:
data = get_data(sz, bs)

In [21]:
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [22]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [0.01, 0.01, 0.1]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.


learn.precompute = False

In [ ]:
learn.fit(1e-5, 100)

epoch      trn_loss   val_loss   accuracy                    
    0      0.70561    0.670311   0.589744  
    1      0.70259    0.669082   0.588738                    
    2      0.710278   0.674025   0.58371                     
    3      0.720694   0.6691     0.589744                    
    4      0.706983   0.672902   0.58371                     
    5      0.716244   0.674016   0.582202                    
    6      0.712162   0.670489   0.584213                    
    7      0.704311   0.669369   0.589241                    
    8      0.708938   0.673322   0.582705                    
    9      0.71751    0.670292   0.583208                    
    10     0.705293   0.670864   0.585219                    
    11     0.712002   0.678431   0.576672                    
    12     0.698858   0.672541   0.581197                    
    13     0.709668   0.67524    0.585219                    
    14     0.70078    0.67197    0.583208                    
    15     0.711332   0.67

In [52]:
learn.save('model-emotion')

In [58]:
learn.load('model-emoation')

FileNotFoundError: [Errno 2] No such file or directory: '/home/embsysa/fastai/pytorch_classifiers-master/data/tmp/256/models/model-emoation.h5'

In [50]:
learn.fit(1e-4,1,best_save_name = 'model1')

epoch      trn_loss   val_loss   accuracy                    
    0      0.713014   0.671208   0.588235  


[0.6712083112600162, 0.5882353123676364]

### Augment

In [28]:
from sklearn import metrics

In [29]:
data = get_data(sz, bs)

In [30]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

100%|██████████| 5/5 [00:00<00:00, 25.09it/s]


In [31]:
learn.fit(1e-2, 100)

epoch      trn_loss   val_loss   accuracy                    
    0      0.933143   0.737388   0.533937  
    1      0.871906   0.735737   0.564605                    
    2      0.811323   0.745391   0.512821                    
    3      0.765969   0.694822   0.576672                    
    4      0.751269   0.696843   0.570639                    
    5      0.753415   0.686339   0.59628                     
    6      0.726065   0.699692   0.54902                     
    7      0.739758   0.684116   0.576672                    
    8      0.728431   0.683846   0.584716                    
    9      0.716244   0.7011     0.543992                    
    10     0.706878   0.678616   0.576169                    
    11     0.676469   0.6708     0.59628                     
    12     0.676268   0.691095   0.562594                    
    13     0.680057   0.68027    0.581197                    
    14     0.67905    0.668335   0.590246                    
    15     0.67916    0.67

[0.7168308818322321, 0.5580693990396698]

In [32]:
learn.save('model2')

In [33]:
learn.precompute = False

In [34]:
learn.fit(1e-2, 100, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                    
    0      0.644201   0.702035   0.558069  
    1      0.602646   0.693725   0.559578                    
    2      0.611055   0.692102   0.570639                    
    3      0.60558    0.689898   0.564103                    
    4      0.602614   0.688689   0.564103                    
    5      0.595946   0.686846   0.567119                    
    6      0.599455   0.690465   0.561086                    
    7      0.598676   0.692953   0.54902                     
    8      0.608997   0.692893   0.559578                    
    9      0.611142   0.690091   0.57265                     
    10     0.606026   0.694665   0.561589                    
    11     0.598044   0.695447   0.565611                    
    12     0.605047   0.695386   0.563097                    
    13     0.599862   0.691376   0.577677                    
    14     0.599632   0.69501    0.566114                    
    15     0.602015   0.69

[0.7102603737944752, 0.5545500428788083]

In [35]:
learn.save('model3')

In [36]:
learn.load('model3')

## Increase size

In [37]:
# Starting training on small images for a few epochs, then switching to bigger images, and continuing training is an amazingly effective way to avoid overfitting.

# http://forums.fast.ai/t/planet-classification-challenge/7824/96
# set_data doesn’t change the model at all. It just gives it new data to train with.
learn.set_data(get_data(299, bs)) 
learn.freeze()

#Source:   
#    def set_data(self, data, precompute=False):
#        super().set_data(data)
#        if precompute:
#            self.unfreeze()
#            self.save_fc1()
#            self.freeze()
#            self.precompute = True
#        else:
#            self.freeze()
#File:      ~/fastai/courses/dl1/fastai/conv_learner.py

In [38]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 299, 299]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('trainable', False),
                           ('nb_params', tensor(9408))])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('trainable', False),
                           ('nb_params', tensor(128))])),
             ('ReLU-3',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('nb_params', 0)])),
             ('MaxPool2d-4',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 75, 75]),
                           ('nb_params', 0)])),
             ('Conv2d-5',
              OrderedDict([('input_shape

In [39]:
learn.fit(1e-2, 100, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                    
    0      0.624505   0.708039   0.564605  
    1      0.604895   0.698083   0.566616                    
    2      0.58685    0.70948    0.544997                    
    3      0.611657   0.70897    0.5455                      
    4      0.605683   0.704298   0.559578                    
    5      0.605451   0.700077   0.566114                    
    6      0.608663   0.697308   0.574661                    
    7      0.600216   0.70359    0.559578                    
    8      0.601635   0.70107    0.556561                    
    9      0.60093    0.699075   0.567622                    
    10     0.589929   0.701714   0.567119                    
    11     0.602794   0.697521   0.574158                    
    12     0.595437   0.697394   0.570136                    
    13     0.586751   0.702555   0.551533                    
    14     0.600834   0.699458   0.565611                    
    15     0.59538    0.70

[0.7175673862208428, 0.5535445105436161]

In [40]:
learn.save('model4')

Validation loss is much lower than training loss. This is a sign of underfitting. Cycle_len=1 may be too short. Let's set cycle_mult=2 to find better parameter.

In [41]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [1e-5, 1e-5, 1e-4]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.

In [43]:
# When you are under fitting, it means cycle_len=1 is too short (learning rate is getting reset before it had the chance to zoom in properly).
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2) # 1+2+4 = 7 epochs

 50%|█████     | 166/332 [00:48<00:49,  3.34it/s, loss=0.73] 

KeyboardInterrupt: 

In [ ]:
learn.save('model5')

Training loss and validation loss are getting closer and smaller. We are on right track.

In [ ]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

In [ ]:
len(data.val_ds.y), data.val_ds.y[:5]

In [ ]:
num = 13
print(data.val_ds.y[num])
print(np.argmax(probs[num]))

print(probs[num])
data.val_ds.fnames[num]

In [ ]:
np.argmax(probs,axis = 1)

In [ ]:
predictions = pd.DataFrame([data.val_ds.y,np.argmax(probs,axis = 1),
                            [each[1] for each in np.argsort(-probs,axis = 1)],
                            data.val_ds.fnames]).T

In [ ]:
np.argmax(probs[1])

In [ ]:
print(probs[0])

In [ ]:
predictions.loc[:,'acc1'] = predictions[0] == predictions[1]
predictions.loc[:,'acc2'] = predictions[0] == predictions[2]
((predictions.acc1) | (predictions.acc2)).sum()

In [ ]:
len(predictions)

In [ ]:
predictions.acc1.sum()/1883

In [ ]:
learn.save('299_pre')

In [ ]:
learn.load('299_pre')

In [ ]:
learn.fit(1e-2, 50, cycle_len=2) # 1+1 = 2 epochs

In [ ]:
learn.save('299_pre')

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

This dataset is so similar to ImageNet dataset. Training convolution layers doesn't help much. We are not going to unfreeze.

## Create submission

https://youtu.be/9C06ZPF8Uuc?t=1905

In [ ]:
data.classes

In [ ]:
data.test_ds.fnames

In [ ]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels

In [ ]:
probs.shape # (n_images, n_classes)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [ ]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]
fn

In [ ]:
Image.open(PATH + fn).resize((150, 150))

In [ ]:
# Method 1.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
learn.data.classes[np.argmax(preds)]

In [ ]:
# Method 2.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
im = val_tfms(open_image(PATH + fn)) # open_image() returns numpy.ndarray
preds = learn.predict_array(im[None])
np.argmax(preds)